# 分类数据
pandas的分类类型能够提高性能和内存的使用率。
# 问题
表中的一列通常会有重复的包含不同值的小集合的情况。我们已经学过了unique和value_counts，它们可以从数组提取出不同的值，并分别计算频率：

In [1]:
import pandas as pd
import tushare as ts

In [ ]:
df=ts.get_stock_basics()
df.head()

In [ ]:
stock=df.loc[:,['industry','area']]
stock.head()

In [ ]:
stock.area.unique()

In [ ]:
stock.area.value_counts()

许多数据系统（数据仓库、统计计算或其它应用）都发展出了特定的表征重复值的方法，以进行高效的存储和计算。在数据仓库中，最好的方法是使用所谓的包含不同值得维表(Dimension Table)，将主要的参数存储为引用维表整数键：

In [ ]:
v=pd.Series([0,1,0,0]*2)
d=pd.Series(['horse','sheep'])
v,d

In [ ]:
d.take(v)

这种用整数表示的方法称为分类或字典编码表示法。不同值得数组称为分类、字典或数据级。表示分类的整数值称为分类编码或简单地称为编码。
分类表示可以在进行分析时大大的提高性能。你也可以在保持编码不变的情况下，对分类进行转换。一些相对简单的转变例子包括：
* 重命名分类。
* 加入一个新的分类，不改变已经存在的分类的顺序或位置。


# pandas的分类类型
pandas有一个特殊的分类类型，用于保存使用整数分类表示法的数据。

In [ ]:
area=df.area.astype('category')
area.head()

In [ ]:
type(area.values)

In [ ]:
c=area.values

In [ ]:
c.categories,c.codes

将DF的列转化为分类类型

In [ ]:
cat=stock.assign(area=stock.area.astype('category'))
cat.head()

In [ ]:
cat.area.head()

# 用分类进行计算
使用pandas.qcut面元函数。它会返回category对象

In [ ]:
bins=pd.qcut(df.pe,6)

In [ ]:
bins.head()

In [ ]:
bins=pd.qcut(df.pe,6,labels=list('abcdef'))
bins.head()

In [ ]:
bins.values.categories,bins.values.codes

加上标签的面元分类不包含数据面元边界的信息，因此可以使用groupby提取一些汇总信息：

In [ ]:
df.pe.groupby(bins).agg(['count','mean'])

# 用分类提高性能
如果你是在一个特定数据集上做大量分析，将其转换为分类可以极大地提高效率。DataFrame列的分类使用的内存通常少的多。

In [ ]:
cat=df.area.astype("category")

In [ ]:
cat.memory_usage(),df.area.memory_usage()

# 为建模创建虚拟变量
当你使用统计或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为one-hot编码。这包括创建一个不同类别的列的DataFrame；这些列包含给定分类的1，其它为0

In [ ]:
pd.get_dummies(df.area.astype('category')).head()

# GroupBy高级应用,分组转换和“解封”GroupBy
我们在分组操作中学习了apply方法，进行转换。还有另一个transform方法，它与apply很像，但是对使用的函数有一定限制：

* 它可以产生向分组形状广播标量值
* 它可以产生一个和输入组形状相同的对象
* 它不能修改输入


In [ ]:
g=df.loc[:,['area','pe']].groupby('area')
g.mean().head()

假设我们想产生一个和df.loc[:,['area','pe']]形状相同的Series，但值替换为按键分组的平均值。我们可以传递函数lambda x: x.mean()进行转换：

In [ ]:
g.transform(lambda x:x.mean()).head()

In [ ]:
g.transform('count').head()

我们可以计算每个分组的降序排名

In [ ]:
g.transform(lambda x:x.rank(ascending=False)).head()

看一个由简单聚合构造的分组转化函数

In [ ]:
g.transform(lambda x:(x-x.mean())/x.std()).head()

我们可以用tansform或者apply可以获得等价结果

In [ ]:
g.apply(lambda x:(x-x.mean())/x.std()).head()

内置的聚合函数，比如mean或sum，通常比apply函数快，也比transform快。这允许我们进行一个所谓的解封（unwrapped）分组操作：

In [ ]:
((df.loc[:,['pe']]-g.transform('mean'))/g.transform('std')).head()

# 股票行业地域热力图

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('whitegrid',{'font.sans-serif':['simhei','Arial']})
a=df.loc[:,['industry','area','pe']]
b=a.groupby(['industry','area']).count()
b.head()

In [ ]:
c=b.reset_index()
c.head()

In [ ]:
d=c.pivot('industry','area','pe')
d.head()

In [ ]:
e=d.sort_values(by=df.area.unique().tolist())
e.head()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(e)

# 管道方法

In [ ]:
plt.figure(figsize=(15,15))
df \
.pipe(lambda x:x.loc[:,['industry','area','pe']]) \
.pipe(lambda x:x.groupby(['industry','area']).count()) \
.pipe(lambda x:x.reset_index().pivot('industry','area','pe').sort_values(by=df.loc[:,'area'].unique().tolist())) \
.pipe(lambda x:sns.heatmap(x))